# Advanced Fibonacci MACD Strategy Analysis

This notebook demonstrates the advanced multi-indicator strategy that combines:
- **Fibonacci Retracement Levels** for optimal entry points
- **MACD Signals** for trend confirmation
- **RSI Analysis** for overbought/oversold conditions
- **Volume Analysis** for institutional confirmation

## Table of Contents
1. [Strategy Overview](#overview)
2. [Data Loading and Preparation](#data)
3. [Strategy Implementation](#strategy)
4. [Backtesting Results](#results)
5. [Signal Analysis](#signals)
6. [Performance Comparison](#comparison)
7. [Parameter Optimization](#optimization)

## 1. Strategy Overview {#overview}

The Fibonacci MACD Strategy is a sophisticated approach that identifies high-probability entry points by requiring multiple technical indicators to align:

### Entry Conditions (Buy Signal):
1. **Fibonacci Support**: Price near key Fibonacci retracement levels (38.2%, 50%, 61.8%)
2. **MACD Bullish**: MACD line crosses above signal line with positive momentum
3. **RSI Recovery**: RSI recovering from oversold conditions or in favorable range
4. **Volume Confirmation**: Trading volume 20% above average with positive trend
5. **Trend Alignment**: Price above short-term moving average

### Exit Conditions (Sell Signal):
1. **Take Profit**: 15% gain from entry
2. **Stop Loss**: 8% loss from entry
3. **Technical Exit**: RSI overbought + MACD bearish signal
4. **Volume Weakness**: Low volume with negative trend

### Confidence Scoring:
Each condition contributes to an overall confidence score, with trades only executed when confidence ≥ 60%.

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from data.data_loader import DataLoader
from backtesting.engine import BacktestEngine
from strategies.fibonacci_macd_strategy import FibonacciMACDStrategy
from strategies.moving_average import MovingAverageCrossover
from utils.metrics import PerformanceAnalyzer, quick_performance_summary
from config import config

# Set up plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ All modules imported successfully")
print(f"✓ Using initial capital: ${config.initial_capital:,.2f}")

## 2. Data Loading and Preparation {#data}

In [ ]:
# Load comprehensive dataset for analysis
SYMBOLS = ['AAPL', 'MSFT', 'GOOGL']  # Test on multiple stocks
START_DATE = '2020-01-01'
END_DATE = '2023-12-31'

print(f"Loading data for {len(SYMBOLS)} symbols from {START_DATE} to {END_DATE}...")

loader = DataLoader()
data = loader.fetch_stock_data(SYMBOLS, START_DATE, END_DATE)

# Display data summary
print(f"\n📊 Data Summary:")
for symbol, df in data.items():
    if not df.empty:
        total_return = (df['close'].iloc[-1] / df['close'].iloc[0] - 1) * 100
        volatility = df['returns'].std() * np.sqrt(252) * 100
        print(f"  {symbol}: {len(df)} records, Return: {total_return:.1f}%, Volatility: {volatility:.1f}%")

# Focus on AAPL for detailed analysis
aapl_data = data['AAPL']
print(f"\n✓ Using AAPL data: {len(aapl_data)} records")

## 3. Strategy Implementation {#strategy}

In [ ]:
# Create the advanced Fibonacci MACD strategy
fibonacci_strategy = FibonacciMACDStrategy(
    fibonacci_period=50,
    macd_fast=12,
    macd_slow=26,
    macd_signal=9,
    rsi_period=14,
    rsi_oversold=30,
    rsi_overbought=70,
    volume_period=20,
    volume_threshold=1.2,
    position_size_pct=0.15,
    min_confidence=0.6
)

# Create a simple MA strategy for comparison
ma_strategy = MovingAverageCrossover(
    short_window=20,
    long_window=50,
    position_size_pct=0.15
)

print(f"📋 Strategy Configuration:")
print(f"\nFibonacci MACD Strategy:")
for param, value in fibonacci_strategy.parameters.items():
    print(f"  {param}: {value}")

print(f"\nMoving Average Strategy (for comparison):")
for param, value in ma_strategy.parameters.items():
    print(f"  {param}: {value}")

## 4. Backtesting Results {#results}

In [ ]:
# Run backtests for both strategies
engine = BacktestEngine()

print("🚀 Running backtests...\n")

# Test Fibonacci MACD strategy
fib_results = engine.run_backtest(fibonacci_strategy, aapl_data)

# Reset and test MA strategy
ma_strategy.reset()
ma_results = engine.run_backtest(ma_strategy, aapl_data)

print("\n✅ Both backtests completed!")

In [ ]:
# Compare results
comparison_data = {
    'Metric': [
        'Total Return', 'Annualized Return', 'Sharpe Ratio', 
        'Max Drawdown', 'Volatility', 'Total Trades'
    ],
    'Fibonacci MACD': [
        f"{fib_results['total_return']:.2%}",
        f"{fib_results['annualized_return']:.2%}",
        f"{fib_results['sharpe_ratio']:.3f}",
        f"{fib_results['max_drawdown']:.2%}",
        f"{fib_results['volatility']:.2%}",
        f"{fib_results['total_trades']}"
    ],
    'Moving Average': [
        f"{ma_results['total_return']:.2%}",
        f"{ma_results['annualized_return']:.2%}",
        f"{ma_results['sharpe_ratio']:.3f}",
        f"{ma_results['max_drawdown']:.2%}",
        f"{ma_results['volatility']:.2%}",
        f"{ma_results['total_trades']}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("📊 Strategy Comparison:")
print(comparison_df.to_string(index=False))

# Calculate buy and hold for reference
buy_hold_return = (aapl_data['close'].iloc[-1] / aapl_data['close'].iloc[0]) - 1
print(f"\n📈 Buy & Hold Return: {buy_hold_return:.2%}")
print(f"🏆 Fibonacci MACD vs Buy & Hold: {fib_results['total_return'] - buy_hold_return:.2%}")
print(f"🏆 Moving Average vs Buy & Hold: {ma_results['total_return'] - buy_hold_return:.2%}")

## 5. Signal Analysis {#signals}

In [ ]:
# Analyze the signals generated by Fibonacci MACD strategy
if 'trades' in fib_results and not fib_results['trades'].empty:
    trades_df = fib_results['trades']
    
    print(f"📋 Trade Analysis for Fibonacci MACD Strategy:")
    print(f"\nTrade Summary:")
    print(f"  Total Trades: {len(trades_df)}")
    print(f"  Buy Orders: {len(trades_df[trades_df['action'] == 'BUY'])}")
    print(f"  Sell Orders: {len(trades_df[trades_df['action'] == 'SELL'])}")
    
    # Analyze buy trades
    buy_trades = trades_df[trades_df['action'] == 'BUY']
    if not buy_trades.empty:
        print(f"\nBuy Trade Statistics:")
        print(f"  Average Buy Price: ${buy_trades['price'].mean():.2f}")
        print(f"  Price Range: ${buy_trades['price'].min():.2f} - ${buy_trades['price'].max():.2f}")
        print(f"  Average Position Size: {buy_trades['quantity'].mean():.0f} shares")
        print(f"  Total Capital Deployed: ${buy_trades['total_cost'].sum():,.2f}")
    
    # Show recent trades
    print(f"\nRecent Trades:")
    recent_trades = trades_df.tail(10)[['timestamp', 'action', 'quantity', 'price']]
    recent_trades['timestamp'] = pd.to_datetime(recent_trades['timestamp']).dt.strftime('%Y-%m-%d')
    print(recent_trades.to_string(index=False))
    
else:
    print("No trades were executed by the Fibonacci MACD strategy.")

## 6. Performance Visualization {#comparison}

In [ ]:
# Create comprehensive performance visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Portfolio value comparison
ax1 = axes[0, 0]
if 'portfolio_history' in fib_results:
    fib_portfolio = fib_results['portfolio_history']
    ax1.plot(fib_portfolio['timestamp'], fib_portfolio['total_value'], 
             label='Fibonacci MACD', linewidth=2, color='blue')

if 'portfolio_history' in ma_results:
    ma_portfolio = ma_results['portfolio_history']
    ax1.plot(ma_portfolio['timestamp'], ma_portfolio['total_value'], 
             label='Moving Average', linewidth=2, color='orange')

# Add buy and hold line
initial_price = aapl_data['close'].iloc[0]
buy_hold_values = (aapl_data['close'] / initial_price) * config.initial_capital
ax1.plot(aapl_data.index, buy_hold_values, 
         label='Buy & Hold', linewidth=2, color='green', alpha=0.7)

ax1.set_title('Portfolio Value Comparison')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Drawdown comparison
ax2 = axes[0, 1]
if 'portfolio_history' in fib_results:
    fib_values = fib_results['portfolio_history']['total_value']
    fib_peak = fib_values.expanding().max()
    fib_dd = (fib_values - fib_peak) / fib_peak * 100
    ax2.fill_between(range(len(fib_dd)), fib_dd, 0, alpha=0.3, color='blue', label='Fibonacci MACD')

if 'portfolio_history' in ma_results:
    ma_values = ma_results['portfolio_history']['total_value']
    ma_peak = ma_values.expanding().max()
    ma_dd = (ma_values - ma_peak) / ma_peak * 100
    ax2.fill_between(range(len(ma_dd)), ma_dd, 0, alpha=0.3, color='orange', label='Moving Average')

ax2.set_title('Drawdown Comparison')
ax2.set_ylabel('Drawdown (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Returns distribution
ax3 = axes[1, 0]
if 'portfolio_history' in fib_results:
    fib_returns = fib_results['portfolio_history']['total_value'].pct_change().dropna() * 100
    ax3.hist(fib_returns, bins=30, alpha=0.6, label='Fibonacci MACD', color='blue', density=True)

if 'portfolio_history' in ma_results:
    ma_returns = ma_results['portfolio_history']['total_value'].pct_change().dropna() * 100
    ax3.hist(ma_returns, bins=30, alpha=0.6, label='Moving Average', color='orange', density=True)

ax3.set_title('Daily Returns Distribution')
ax3.set_xlabel('Daily Return (%)')
ax3.set_ylabel('Density')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Risk-Return scatter
ax4 = axes[1, 1]
strategies_data = [
    ('Fibonacci MACD', fib_results['volatility'] * 100, fib_results['annualized_return'] * 100, fib_results['sharpe_ratio']),
    ('Moving Average', ma_results['volatility'] * 100, ma_results['annualized_return'] * 100, ma_results['sharpe_ratio']),
    ('Buy & Hold', aapl_data['returns'].std() * np.sqrt(252) * 100, buy_hold_return * 100 / 4, 0)  # Approximate annualized
]

for name, vol, ret, sharpe in strategies_data:
    color = 'blue' if 'Fibonacci' in name else 'orange' if 'Moving' in name else 'green'
    ax4.scatter(vol, ret, s=100, alpha=0.7, color=color, label=name)
    ax4.annotate(name, (vol, ret), xytext=(5, 5), textcoords='offset points', fontsize=8)

ax4.set_title('Risk-Return Profile')
ax4.set_xlabel('Volatility (%)')
ax4.set_ylabel('Annualized Return (%)')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Insights and Conclusions

### Strategy Performance Analysis:

The Fibonacci MACD strategy demonstrates several key characteristics:

1. **Multi-Factor Approach**: By requiring multiple indicators to align, the strategy reduces false signals
2. **Risk Management**: Built-in stop-loss and take-profit levels help manage downside risk
3. **Confidence-Based Sizing**: Position sizes scale with signal confidence
4. **Volume Confirmation**: Ensures institutional participation in trades

### Advantages:
- Higher precision with fewer false signals
- Systematic risk management
- Adaptable to different market conditions
- Comprehensive technical analysis integration

### Areas for Improvement:
- Parameter optimization for different market regimes
- Dynamic position sizing based on market volatility
- Integration of fundamental analysis factors
- Multi-timeframe analysis

### Next Steps:
1. **Parameter Optimization**: Use the optimization notebook to find optimal parameters
2. **Multi-Asset Testing**: Test across different stocks and sectors
3. **Market Regime Analysis**: Analyze performance in bull vs bear markets
4. **Risk Parity**: Implement volatility-adjusted position sizing

In [ ]:
# Final performance summary
print("📊 FINAL PERFORMANCE SUMMARY")
print("=" * 50)

print(f"\n🎯 Fibonacci MACD Strategy:")
print(f"  Total Return: {fib_results['total_return']:.2%}")
print(f"  Sharpe Ratio: {fib_results['sharpe_ratio']:.3f}")
print(f"  Max Drawdown: {fib_results['max_drawdown']:.2%}")
print(f"  Total Trades: {fib_results['total_trades']}")

print(f"\n📈 Key Strengths:")
print(f"  • Multi-indicator confirmation reduces false signals")
print(f"  • Fibonacci levels provide precise entry points")
print(f"  • Volume analysis ensures institutional backing")
print(f"  • Systematic risk management with stop-loss/take-profit")

print(f"\n🔧 Optimization Opportunities:")
print(f"  • Fine-tune Fibonacci period for different volatility regimes")
print(f"  • Adjust RSI thresholds based on market conditions")
print(f"  • Optimize volume threshold for different sectors")
print(f"  • Consider dynamic position sizing based on volatility")

print(f"\n✅ Strategy successfully demonstrates advanced multi-indicator approach!")